# 1. 설정 및 초기화

## 라이브러리 설치 및 임포트

In [1]:
!unzip /content/drive/MyDrive/pdf_files.zip
!pip install langchain pypdf tiktoken openai

import os
import json
import re
import hashlib
import logging
import time
import tiktoken
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
from functools import lru_cache
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai.types.chat import ChatCompletion
import tiktoken

Archive:  /content/drive/MyDrive/pdf_files.zip
   creating: pdf_files/
  inflating: pdf_files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf  
  inflating: pdf_files/(사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf  
  inflating: pdf_files/(사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf  
  inflating: pdf_files/(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf  
  inflating: pdf_files/2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf  
  inflating: pdf_files/BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf  
  inflating: pdf_files/KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf  
  inflating: pdf_files/경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf  
  inflating: pdf_files/경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf  
  inflating: pdf_files/경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf  
  inflating: pdf_files/경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf  
  inflating: pdf_files/경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf  
  inflating: pdf_files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf  
  inflating: pdf_files/고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운

## 로깅 설정

In [2]:
"""
# 환경변수 로드
load_dotenv()
"""

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 2. 데이터 준비

## PDF 로딩 함수

In [3]:
def load_pdf(file_path: str) -> str:
    """PDF 파일에서 텍스트 전체를 추출합니다."""
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or "" # 텍스트가 없는 페이지는 스킵
        return text
    except Exception as e:
        print(f"PDF 로딩 오류 ({os.path.basename(file_path)}): {e}")
        return ""

## 텍스트 청킹

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # 1000 글자 단위로 자르기
    chunk_overlap=100,  # 100 글자씩 겹치게 자르기 (문맥 유지)
    length_function=len,
    is_separator_regex=False,
)

## 파일 처리 실행

In [5]:
# PDF 파일이 있는 폴더 경로
file_directory = "/content/pdf_files/"

# RAG의 핵심 재료: 모든 청크(Chunk)를 저장할 리스트
all_chunks = []
processed_file_count = 0

# 지정된 디렉토리가 존재하는지 확인
if not os.path.exists(file_directory):
    print(f"폴더를 찾을 수 없습니다: {file_directory}")
else:
    # 폴더 내의 모든 파일을 순회
    for file_name in os.listdir(file_directory):

        # .pdf 파일만 대상으로 함
        if file_name.endswith(".pdf"):
            file_path = os.path.join(file_directory, file_name)

            # 1. Loading: PDF에서 텍스트 추출
            print(f"[PDF] 로딩 중: {file_name}")
            raw_text = load_pdf(file_path)

            if not raw_text:
                print(f"텍스트 추출 실패: {file_name}")
                continue

            # 2. Chunking: 긴 텍스트를 청크로 분할
            chunks = text_splitter.split_text(raw_text)

            print(f"  -> {file_name}에서 {len(chunks)}개의 청크 생성됨.")

            # 3. 메타데이터 추가
            for i, chunk_text in enumerate(chunks):
                all_chunks.append({
                    "text": chunk_text,
                    "metadata": {
                        "source_id": file_name, # Generator가 출처 표시에 사용
                        "chunk_index": i
                    }
                })

            processed_file_count += 1

print(f"총 {processed_file_count}개 PDF에서 {len(all_chunks)}개 청크 생성")

[PDF] 로딩 중: 축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).pdf
  -> 축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).pdf에서 107개의 청크 생성됨.
[PDF] 로딩 중: 한국사회보장정보원_라오스 보건의료정보화 협력을 위한 사전타당성 조.pdf
  -> 한국사회보장정보원_라오스 보건의료정보화 협력을 위한 사전타당성 조.pdf에서 71개의 청크 생성됨.
[PDF] 로딩 중: 수협중앙회_강릉어선안전조업국 상황관제시스템 구축.pdf
  -> 수협중앙회_강릉어선안전조업국 상황관제시스템 구축.pdf에서 70개의 청크 생성됨.
[PDF] 로딩 중: 한국발명진흥회 입찰공고_2024년 건설기술에 관한 특허·실용신안 활용실.pdf
  -> 한국발명진흥회 입찰공고_2024년 건설기술에 관한 특허·실용신안 활용실.pdf에서 73개의 청크 생성됨.
[PDF] 로딩 중: 국방과학연구소_대용량 자료전송시스템 고도화.pdf
  -> 국방과학연구소_대용량 자료전송시스템 고도화.pdf에서 72개의 청크 생성됨.
[PDF] 로딩 중: 세종테크노파크_세종테크노파크 인사정보 전산시스템 구축 용역 입찰공.pdf
  -> 세종테크노파크_세종테크노파크 인사정보 전산시스템 구축 용역 입찰공.pdf에서 106개의 청크 생성됨.
[PDF] 로딩 중: 한국사학진흥재단_대학재정정보시스템(기본재산 및 기채 사후관리) 고.pdf
  -> 한국사학진흥재단_대학재정정보시스템(기본재산 및 기채 사후관리) 고.pdf에서 138개의 청크 생성됨.
[PDF] 로딩 중: 대한적십자사 의료원_적십자병원 병원정보 재해복구시스템 구축 용역 .pdf
  -> 대한적십자사 의료원_적십자병원 병원정보 재해복구시스템 구축 용역 .pdf에서 61개의 청크 생성됨.
[PDF] 로딩 중: 서울특별시교육청_서울특별시교육청 지능정보화전략계획(ISP) 수립(2차) .pdf
  -> 서울특별시교육청_서울특별시교육청 지능정보화전략계획(ISP) 수립(2차) .pdf에서 81개의 청크 생성됨.
[PDF] 로딩 중: 경희대

## 모델 및 생성 설정 최적화

In [20]:
class ModelConfig:
    """비용과 성능을 고려한 모델 설정"""

    # 모델별 특성
    MODELS = {
        "gpt-5": {
            "cost_per_1k_input": 0.0025,
            "cost_per_1k_output": 0.010,
            "context_window": 128000,
            "best_for": "복잡한 입찰 분석"
        },
        "gpt-5-mini": {
            "cost_per_1k_input": 0.00015,
            "cost_per_1k_output": 0.0006,
            "context_window": 128000,
            "best_for": "일반 질의응답"
        },
        "gpt-5-nano": {
            "cost_per_1k_input": 0.0005,
            "cost_per_1k_output": 0.0015,
            "context_window": 16385,
            "best_for": "간단한 정보 추출"
        }
    }

    @classmethod
    def get_optimal_model(cls, query_complexity: str = "medium") -> str:
        """쿼리 복잡도에 따른 최적 모델 선택"""
        if query_complexity == "high":
            return "gpt-5"
        elif query_complexity == "low":
            return "gpt-5-nano"
        else:
            return "gpt-5-mini"  # 기본값: 비용 대비 성능 최적

    @classmethod
    def get_generation_config(cls, response_type: str = "detailed") -> Dict[str, Any]:
        """응답 타입별 최적화된 생성 설정"""
        configs = {
            "detailed": {
                "max_completion_tokens": 1500
            },
            "concise": {
                "max_completion_tokens": 500
            },
            "creative": {
                "max_completion_tokens": 1000
            }
        }
        return configs.get(response_type, configs["detailed"])

## 프롬프트 최적화

In [7]:
class OptimizedPrompts:
    """토큰 사용량을 최적화한 프롬프트 템플릿"""

    SYSTEM_PROMPT = """당신은 '입찰메이트'의 B2G 입찰 전문 AI 어시스턴트입니다.

**핵심 원칙:**
1. 제공된 [참고 문서]의 내용만 사용
2. 문서에 없는 내용은 "문서에서 확인할 수 없습니다" 명시
3. 숫자, 날짜, 금액은 정확히 인용
4. 간결하고 구조화된 답변

**답변 형식:**
- 핵심 내용을 bullet point로 정리
- 중요 키워드는 **볼드** 처리
- 애매한 표현 지양

**톤:** 전문적이지만 이해하기 쉽게"""

    @staticmethod
    def create_user_prompt(query: str, chunks: List[Dict], chat_history: Optional[List[Dict]] = None) -> str:
        """토큰 효율적인 사용자 프롬프트 생성"""

        # 컨텍스트 구성 (최소한의 메타데이터만 포함)
        context_parts = []
        seen_sources = set()

        for i, chunk in enumerate(chunks, 1):
            source_id = chunk.get('metadata', {}).get('source_id', f'문서{i}')
            text = chunk.get('text', '').strip()

            # 중복 소스 표시 최소화
            if source_id not in seen_sources:
                context_parts.append(f"[{source_id}]")
                seen_sources.add(source_id)

            context_parts.append(text)
            context_parts.append("")  # 청크 간 구분

        context_str = "\n".join(context_parts)

        # 대화 히스토리 요약 (있는 경우)
        history_str = ""
        if chat_history and len(chat_history) > 0:
            # 최근 2턴만 포함하여 토큰 절약
            recent_history = chat_history[-4:]  # user + assistant 2쌍
            history_lines = []
            for msg in recent_history:
                role = "사용자" if msg["role"] == "user" else "AI"
                content = msg["content"][:100]  # 100자로 제한
                history_lines.append(f"{role}: {content}")
            history_str = "\n이전 대화:\n" + "\n".join(history_lines) + "\n\n"

        # 최종 프롬프트 (토큰 절약형)
        return f"""{history_str}[참고 문서]
{context_str}

질문: {query}

위 문서를 바탕으로 답변해주세요."""

    @staticmethod
    def estimate_tokens(text: str, model: str = "gpt-5-mini") -> int:
        """텍스트의 토큰 수 추정"""
        try:
            encoding = tiktoken.encoding_for_model(model)
        except KeyError:
            encoding = tiktoken.get_encoding("cl100k_base")
        return len(encoding.encode(text))

## 토큰 사용량 최적화

In [8]:
class TokenOptimizer:
    """토큰 사용량 최적화 유틸리티"""

    @staticmethod
    def truncate_chunks_smart(
        chunks: List[Dict],
        query: str,
        max_tokens: int = 4000,
        model: str = "gpt-5-mini"
    ) -> List[Dict]:
        """쿼리 관련성과 토큰 제한을 고려한 청크 선택"""

        # 간단한 키워드 기반 관련성 스코어링
        query_keywords = set(query.lower().split())

        scored_chunks = []
        for chunk in chunks:
            text = chunk.get('text', '').lower()
            # 키워드 매칭 스코어
            matches = sum(1 for kw in query_keywords if kw in text)
            scored_chunks.append((matches, chunk))

        # 관련성 높은 순으로 정렬
        scored_chunks.sort(key=lambda x: x[0], reverse=True)

        # 토큰 제한까지 선택
        selected = []
        total_tokens = 0
        encoding = tiktoken.encoding_for_model(model)

        for score, chunk in scored_chunks:
            chunk_tokens = len(encoding.encode(chunk['text']))
            if total_tokens + chunk_tokens <= max_tokens:
                selected.append(chunk)
                total_tokens += chunk_tokens
            else:
                break

        logger.info(f"토큰 최적화: {len(chunks)}개 중 {len(selected)}개 선택 ({total_tokens} tokens)")
        return selected

## 대화 관리자

In [9]:
class EnhancedConversationManager:
    """대화 맥락을 유지하는 대화 관리자"""

    def __init__(self, max_history: int = 3, max_tokens_per_turn: int = 300):
        self.history = []
        self.max_history = max_history
        self.max_tokens_per_turn = max_tokens_per_turn
        self.conversation_id = hashlib.md5(str(time.time()).encode()).hexdigest()[:8]

    def add_turn(self, user_msg: str, assistant_msg: str, sources: List[str] = None):
        """대화 턴 추가 (토큰 제한 적용)"""

        # 토큰 제한을 위해 메시지 요약
        user_msg_truncated = self._truncate_message(user_msg)
        assistant_msg_truncated = self._truncate_message(assistant_msg)

        self.history.append({
            "role": "user",
            "content": user_msg_truncated,
            "timestamp": datetime.now().isoformat(),
            "full_content": user_msg  # 필요시 참조용
        })

        self.history.append({
            "role": "assistant",
            "content": assistant_msg_truncated,
            "sources": sources or [],
            "timestamp": datetime.now().isoformat()
        })

        # 히스토리 길이 제한
        if len(self.history) > self.max_history * 2:
            self.history = self.history[-(self.max_history * 2):]

        logger.info(f"대화 턴 추가 (총 {len(self.history)//2}턴)")

    def _truncate_message(self, message: str) -> str:
        """메시지를 토큰 제한에 맞게 축약"""
        encoding = tiktoken.get_encoding("cl100k_base")
        tokens = encoding.encode(message)

        if len(tokens) > self.max_tokens_per_turn:
            truncated_tokens = tokens[:self.max_tokens_per_turn]
            return encoding.decode(truncated_tokens) + "..."
        return message

    def get_history_for_prompt(self) -> List[Dict[str, str]]:
        """프롬프트에 포함할 히스토리 반환 (간결화)"""
        return [
            {"role": msg["role"], "content": msg["content"]}
            for msg in self.history
        ]

    def get_context_summary(self) -> str:
        """대화 맥락 요약"""
        if not self.history:
            return "새로운 대화"

        user_turns = [msg for msg in self.history if msg["role"] == "user"]
        return f"{len(user_turns)}개 질문 진행 중"

    def clear_history(self):
        """대화 히스토리 초기화"""
        self.history = []
        logger.info(f"대화 {self.conversation_id} 히스토리 초기화")

## 프롬프트 템플릿 정의

In [ ]:
class RAGGenerator:
    """최적화된 RAG Generator"""

    def __init__(
        self,
        model: Optional[str] = None,
        response_type: str = "detailed"
    ):
        # API 키 확인
        #api_key = "os.getenv("OPENAI_API_KEY")"
        api_key = "쓰고 지우십쇼"
        if not api_key:
            raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

        self.client = OpenAI(api_key=api_key)
        self.model = model or ModelConfig.get_optimal_model("medium")
        self.config = ModelConfig.get_generation_config(response_type)
        self.conversation_manager = EnhancedConversationManager()
        self.prompts = OptimizedPrompts()
        self.token_optimizer = TokenOptimizer()

        # 메트릭 추적
        self.metrics = {
            "total_queries": 0,
            "total_tokens": 0,
            "total_cost": 0.0,
            "avg_latency_ms": 0.0
        }

        logger.info(f"Generator 초기화: model={self.model}, config={self.config}")

    def generate(
        self,
        query: str,
        retrieved_chunks: List[Dict],
        use_history: bool = True
    ) -> Dict[str, Any]:
        """최적화된 답변 생성"""

        start_time = time.time()
        self.metrics["total_queries"] += 1

        try:
            # 1. 토큰 최적화: 청크 선택
            optimized_chunks = self.token_optimizer.truncate_chunks_smart(
                chunks=retrieved_chunks,
                query=query,
                max_tokens=4000,
                model=self.model
            )

            # 2. 프롬프트 생성
            chat_history = self.conversation_manager.get_history_for_prompt() if use_history else None

            messages = [
                {"role": "system", "content": self.prompts.SYSTEM_PROMPT},
            ]

            if chat_history:
                messages.extend(chat_history)

            user_prompt = self.prompts.create_user_prompt(
                query=query,
                chunks=optimized_chunks,
                chat_history=chat_history
            )
            messages.append({"role": "user", "content": user_prompt})

            # 토큰 수 추정
            estimated_tokens = sum(
                self.prompts.estimate_tokens(msg["content"], self.model)
                for msg in messages
            )
            logger.info(f"입력 토큰 추정: {estimated_tokens}")

            # 3. API 호출
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                **self.config
            )

            # 4. 결과 처리
            answer = completion.choices[0].message.content or "답변을 생성하지 못했습니다."
            answer = self._post_process(answer)

            # 5. 메트릭 업데이트
            tokens_used = completion.usage.total_tokens if completion.usage else 0
            latency_ms = (time.time() - start_time) * 1000
            cost = self._calculate_cost(completion.usage, self.model)

            self.metrics["total_tokens"] += tokens_used
            self.metrics["total_cost"] += cost
            self.metrics["avg_latency_ms"] = (
                (self.metrics["avg_latency_ms"] * (self.metrics["total_queries"] - 1) + latency_ms)
                / self.metrics["total_queries"]
            )

            # 6. 소스 추출
            sources = list(set(
                chunk.get('metadata', {}).get('source_id', 'Unknown')
                for chunk in optimized_chunks
            ))

            # 7. 대화 히스토리 업데이트
            if use_history:
                self.conversation_manager.add_turn(query, answer, sources)

            # 8. 최종 응답
            response = {
                "answer": answer,
                "sources": sorted(sources),
                "metadata": {
                    "model": self.model,
                    "tokens_used": tokens_used,
                    "latency_ms": round(latency_ms, 2),
                    "cost_usd": round(cost, 6),
                    "chunks_used": len(optimized_chunks),
                    "conversation_turns": len(self.conversation_manager.history) // 2
                }
            }

            logger.info(f"답변 생성 완료: {tokens_used} tokens, {latency_ms:.0f}ms, ${cost:.6f}")
            return response

        except Exception as e:
            logger.error(f"답변 생성 실패: {e}")
            return {
                "answer": "죄송합니다. 답변 생성 중 오류가 발생했습니다.",
                "sources": [],
                "metadata": {"error": str(e)}
            }

    def _post_process(self, answer: str) -> str:
        """답변 후처리"""
        # 과도한 줄바꿈 제거
        answer = re.sub(r'\n{3,}', '\n\n', answer)
        # 앞뒤 공백 제거
        answer = answer.strip()
        return answer

    def _calculate_cost(self, usage: Any, model: str) -> float:
        """API 사용 비용 계산"""
        if not usage:
            return 0.0

        model_info = ModelConfig.MODELS.get(model, ModelConfig.MODELS["gpt-5-mini"])
        input_cost = (usage.prompt_tokens / 1000) * model_info["cost_per_1k_input"]
        output_cost = (usage.completion_tokens / 1000) * model_info["cost_per_1k_output"]
        return input_cost + output_cost

    def get_metrics(self) -> Dict[str, Any]:
        """현재까지의 사용 메트릭 반환"""
        return self.metrics.copy()

    def reset_conversation(self):
        """대화 초기화"""
        self.conversation_manager.clear_history()

## 사용 예시

In [22]:
if __name__ == "__main__":
    # Generator 초기화
    generator = RAGGenerator(
        model="gpt-5-mini",  # 비용 대비 성능 최적
        response_type="detailed"
    )

    # 테스트 데이터
    mock_chunks = [
        {
            "text": "국민연금공단 이러닝시스템은 SCORM v1.2 표준을 준수해야 합니다. 콘텐츠 개발 관리 요구사항으로 모든 학습 콘텐츠는 SCORM 1.2 표준을 따라야 하며...",
            "metadata": {"source_id": "국민연금공단_RFP.pdf", "chunk_index": 3}
        },
        {
            "text": "시스템은 1일 최대 5,000명의 동시 접속자를 처리할 수 있어야 하며, 응답 시간은 3초 이내여야 합니다. 또한 99.9% 이상의 가용성을 보장해야 합니다.",
            "metadata": {"source_id": "국민연금공단_RFP.pdf", "chunk_index": 5}
        }
    ]

    # 첫 번째 질문
    print("=" * 80)
    print("질문 1: 국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.")
    print("=" * 80)

    response1 = generator.generate(
        query="국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.",
        retrieved_chunks=mock_chunks
    )

    print(f"\n답변:\n{response1['answer']}")
    print(f"\n출처: {', '.join(response1['sources'])}")
    print(f"\n메타데이터: {json.dumps(response1['metadata'], indent=2, ensure_ascii=False)}")

    # 후속 질문 (대화 맥락 유지)
    print("\n" + "=" * 80)
    print("질문 2: 동시 접속자 수는 몇 명인가요?")
    print("=" * 80)

    response2 = generator.generate(
        query="동시 접속자 수는 몇 명인가요?",
        retrieved_chunks=mock_chunks,
        use_history=True  # 대화 맥락 사용
    )

    print(f"\n답변:\n{response2['answer']}")
    print(f"\n출처: {', '.join(response2['sources'])}")
    print(f"\n메타데이터: {json.dumps(response2['metadata'], indent=2, ensure_ascii=False)}")

    # 답을 모르는 질문 (대화 맥락 유지)
    print("\n" + "=" * 80)
    print("질문 3: 최귀빈이 좋아하는 음식은 무엇인가요?")
    print("=" * 80)

    response3 = generator.generate(
        query="최귀빈이 좋아하는 음식은 무엇인가요?",
        retrieved_chunks=mock_chunks,
        use_history=True  # 대화 맥락 사용
    )

    print(f"\n답변:\n{response3['answer']}")
    print(f"\n출처: {', '.join(response3['sources'])}")
    print(f"\n메타데이터: {json.dumps(response3['metadata'], indent=2, ensure_ascii=False)}")

    # 전체 메트릭 확인
    print("\n" + "=" * 80)
    print("전체 사용 메트릭")
    print("=" * 80)
    metrics = generator.get_metrics()
    print(json.dumps(metrics, indent=2, ensure_ascii=False))

질문 1: 국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.

답변:
- 핵심 요구사항 (문서에 명시된 내용)
  - **콘텐츠 표준**: 모든 학습 콘텐츠는 **SCORM 1.2** 표준을 준수해야 함 ("콘텐츠 개발 관리 요구사항으로 모든 학습 콘텐츠는 SCORM 1.2 표준을 따라야 하며...")
  - **동시 접속자 처리 능력**: 시스템은 **1일 최대 5,000명**의 **동시 접속자**를 처리할 수 있어야 함 ("시스템은 1일 최대 5,000명의 동시 접속자를 처리할 수 있어야 하며")
  - **응답 시간**: 평균/최대 응답 시간은 **3초 이내**여야 함 ("응답 시간은 3초 이내여야 합니다")
  - **가용성**: 시스템 가용성은 **99.9% 이상**을 보장해야 함 ("또한 99.9% 이상의 가용성을 보장해야 합니다")

- 문서에서 확인할 수 없는(추가 확인 필요) 주요 항목
  - **인증·권한 관리(로그인 방식, SSO 등)**: 문서에서 확인할 수 없습니다.
  - **보안 요건(암호화, 침해대응, 인증 기준 등)**: 문서에서 확인할 수 없습니다.
  - **백업·재해복구(RTO/RPO)**: 문서에서 확인할 수 없습니다.
  - **호스팅 방식(온프레미스/클라우드/하이브리드)**: 문서에서 확인할 수 없습니다.
  - **지원되는 브라우저·기기 및 반응형/모바일 지원 범위**: 문서에서 확인할 수 없습니다.
  - **접근성(예: WCAG) 및 다국어 지원**: 문서에서 확인할 수 없습니다.
  - **학습 이력 추적/통계 항목(보고서 항목, 로그 보관 기간 등)**: 문서에서 확인할 수 없습니다.
  - **통합 요구사항(API/연동(인사/전산) 등)**: 문서에서 확인할 수 없습니다.
  - **성능 보증 상세(동시접속 정의·피크 기간·부하분산 방식 등)**: 문서에서 확인할 수 없습니다.
  - **계약·운영 SLA의 세부 항목(유지보수, 지원시간, 페널티 등)**: 문서에서 확인할 수 없습니다.
  